In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
import gensim
import re
import sqlite3
import pickle

C:\Users\Mudit\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
#Load data from the sqlite database file into a Pandas dataframe
cnx = sqlite3.connect('Data/gutenberg/gutenberg.sqlite3')

df = pd.read_sql_query("SELECT * FROM articles", cnx)

In [4]:
#Drop index column

df.drop(columns=['id'],inplace=True)

In [5]:
df['article'] = df['article'].map(lambda x: re.sub('\r|\n|\'','',x))
df['article'] = df['article'].map(lambda x: re.sub(r'--\d\d\d-\d\d\d-\d\d\d\d','',x))
df['article'] = df['article'].map(lambda x: re.sub('\s+',' ',x))

In [5]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words_subset = set([x for x in stop_words if 3 <= len(x) <= 5])

In [6]:
def remove_stopwords(x): #x is the string input
    word_set = x.split(' ')
    for word in word_set:
        if word in stop_words:
            word_set.remove(word)
    return ' '.join(word_set)
            


In [7]:
# Removing stopwords
df['article'] = df['article'].map(lambda x: remove_stopwords(x))

In [25]:
df['auth_name'].iloc[100]

'Twain, Mark'

In [26]:
#Apply tf-idf vectorization on the article column

In [8]:
corpus = str()
for row in df['article'].iteritems():
    corpus += row[1]

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus.split(' ')) #Tfidf Vectorizer accepts a list of strings as input

In [10]:
X = vectorizer.transform(list(df['article']))

In [11]:
y = df['auth_name']

In [12]:
#Applying label binarizer to get one-hot encoding of the categorical text column 'auth_name'

from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
encoder.fit(df['auth_name'].unique())

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [13]:
y = encoder.transform(y)

In [15]:
#Perform train-test split of the dataset:
#Set the random_state attribute to some value so that train-test split returns the same training and testing subsets each time
#Setting the stratify attribute ensures that there are instances of all classes(target features) in both train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.25,
                                                   random_state = 42)

In [131]:
#Apply a simple neural network on this dataset:

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [23]:
input_dimension = X_train.shape[1] #This specifies the number of neurons in the input layer, in this case 444
#Using Keras' sequential API
model = Sequential()
model.add(layers.Dense(100, input_dim=input_dimension, activation='relu'))
model.add(layers.Dense(42, activation='softmax'))

In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
#Fitting the data:
history = model.fit(X_train, y_train,
                     epochs=100,
                     verbose=True,
                     validation_data=(X_test, y_test),
                    batch_size=10)

Train on 1575 samples, validate on 525 samples
Epoch 1/100
1575/1575 [==============================] - 34s 21ms/step - loss: 3.6358 - acc: 0.1276 - val_loss: 3.4635 - val_acc: 0.4267
Epoch 2/100
1575/1575 [==============================] - 20s 13ms/step - loss: 2.8815 - acc: 0.8762 - val_loss: 2.7669 - val_acc: 0.7486
Epoch 3/100
1575/1575 [==============================] - 20s 13ms/step - loss: 1.5198 - acc: 0.9917 - val_loss: 1.9428 - val_acc: 0.8895
Epoch 4/100
1575/1575 [==============================] - 20s 13ms/step - loss: 0.4966 - acc: 0.9968 - val_loss: 1.4939 - val_acc: 0.9067
Epoch 5/100
1575/1575 [==============================] - 20s 13ms/step - loss: 0.1765 - acc: 0.9975 - val_loss: 1.2867 - val_acc: 0.9048
Epoch 6/100
1575/1575 [==============================] - 20s 13ms/step - loss: 0.0887 - acc: 0.9975 - val_loss: 1.1597 - val_acc: 0.9181
Epoch 7/100
1575/1575 [==============================] - 20s 13ms/step - loss: 0.0565 - acc: 0.9975 - val_loss: 1.0738 - val_acc: 0

1575/1575 [==============================] - 19s 12ms/step - loss: 0.0047 - acc: 0.9981 - val_loss: 0.4458 - val_acc: 0.9371
Epoch 61/100
1575/1575 [==============================] - 19s 12ms/step - loss: 0.0047 - acc: 0.9981 - val_loss: 0.4444 - val_acc: 0.9352
Epoch 62/100
1575/1575 [==============================] - 19s 12ms/step - loss: 0.0047 - acc: 0.9981 - val_loss: 0.4384 - val_acc: 0.9410
Epoch 63/100
1575/1575 [==============================] - 19s 12ms/step - loss: 0.0048 - acc: 0.9981 - val_loss: 0.4322 - val_acc: 0.9390
Epoch 64/100
1575/1575 [==============================] - 19s 12ms/step - loss: 0.0047 - acc: 0.9981 - val_loss: 0.4316 - val_acc: 0.9352
Epoch 65/100
1575/1575 [==============================] - 19s 12ms/step - loss: 0.0047 - acc: 0.9981 - val_loss: 0.4264 - val_acc: 0.9429
Epoch 66/100
1575/1575 [==============================] - 19s 12ms/step - loss: 0.0047 - acc: 0.9981 - val_loss: 0.4238 - val_acc: 0.9352
Epoch 67/100
1575/1575 [=======================

In [27]:
model.evaluate(X_train,y_train)

1575/1575 [==============================] - 2s 1ms/step


[0.004241822601682124, 0.9980952380952381]

In [28]:
model.evaluate(X_test,y_test)

525/525 [==============================] - 1s 2ms/step


[0.3388830413704827, 0.9447619047619048]

In [34]:
#pickle.dump(model,open('Pickled models/gutenberg_tfidf_one_hidden','wb'))

In [35]:
dir(model)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_inbound_node',
 '_add_unique_metric_name',
 '_add_variable_with_custom_getter',
 '_assert_input_compatibility',
 '_base_init',
 '_build_input_shape',
 '_cache_output_metric_attributes',
 '_call_and_compute_mask',
 '_call_convention',
 '_check_trainable_weights_consistency',
 '_checkpoint_dependencies',
 '_checkpointable_saver',
 '_collected_trainable_weights',
 '_compute_output_and_mask_jointly',
 '_compute_previous_mask',
 '_dataset_iterator_cache',
 '_deferred_dependencies',
 '_determine_call_convention',
 '_distribution_standardize_user_data',
 '_distributi

In [124]:
X_test

<525x91374 sparse matrix of type '<class 'numpy.float64'>'
	with 230767 stored elements in Compressed Sparse Row format>

In [80]:
vectorizer2 = TfidfVectorizer()
X2 = vectorizer2.fit_transform(corpus.split('.'))

In [86]:
X2

<102195x91374 sparse matrix of type '<class 'numpy.float64'>'
	with 1360640 stored elements in Compressed Sparse Row format>

In [97]:
corpus[:1000]

' With mind thus full happiness, Catherine hardly aware two three days passed away, without seeing Isabella minutes together. She began first sensible this, sigh conversation, walked along pump-room one morning, Mrs. Allens side, without anything say hear; scarcely felt five minutes longing friendship, object appeared, inviting secret conference, led way seat. “This favourite place,” said sat bench doors, commanded tolerable view everybody entering either; “it way.” Catherine, observing Isabellas eyes continually bent towards one door other, eager expectation, remembering often she falsely accused arch, thought present fine opportunity really so; therefore gaily said, “Do uneasy, Isabella, James soon here.” “Psha! My dear creature,” she replied, “do think such simpleton always wanting confine elbow. It would hideous always together; jest place. And going Northanger! I amazingly glad it. It one finest old places England, I understand. I shall depend upon particular description it.” “You

In [109]:
help(encoder.fit_transform)

Help on method fit_transform in module sklearn.preprocessing.label:

fit_transform(y) method of sklearn.preprocessing.label.LabelBinarizer instance
    Fit label binarizer and transform multi-class labels to binary
    labels.
    
    The output of transform is sometimes referred to    as
    the 1-of-K coding scheme.
    
    Parameters
    ----------
    y : array or sparse matrix of shape [n_samples,] or             [n_samples, n_classes]
        Target values. The 2-d matrix should only contain 0 and 1,
        represents multilabel classification. Sparse matrix can be
        CSR, CSC, COO, DOK, or LIL.
    
    Returns
    -------
    Y : array or CSR matrix of shape [n_samples, n_classes]
        Shape will be [n_samples, 1] for binary problems.

